In [ ]:
# !pip install gluonts pytorch-lightning
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.torch import DeepAREstimator
from gluonts.torch.model.deepar import DeepARModel
from gluonts.torch.distributions import StudentTOutput
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
import numpy as np
from pytorch_lightning.callbacks import EarlyStopping

class ProbDeepAR(DeepARModel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Add sigmoid layer after base model's output
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, *args, **kwargs):
        # Get original outputs
        outputs = super().forward(*args, **kwargs)
        
        # Apply sigmoid to mean (while preserving distribution)
        outputs.predicted_mean = self.sigmoid(outputs.predicted_mean)
        return outputs

# Load and preprocess data
data = pd.read_csv('train.csv')
data = data.set_index('day')
data = data.fillna(method='ffill')

# Create GluonTS datasets
k = 2 * 365  # Validation length
data.index = pd.date_range(
    start="2020-01-01",  # Choose any start date
    periods=len(data),
    freq="D"  # Daily frequency
)
train_data = PandasDataset(
    data[:-k],
    target="rainfall",
    freq="D"
)

test_data = PandasDataset(data[-k:], target="rainfall", freq='D')

# Define estimator with modern parameters
estimator = DeepAREstimator(
    freq="D",
    prediction_length=1,
    # model=ProbDeepAR,  # Use custom model with sigmoid activation
    context_length=30,
    distr_output=StudentTOutput(),
    trainer_kwargs={
        "max_epochs": 50,
        "enable_progress_bar": True,
    },
    lr=1e-3,
    num_layers=2,
    hidden_size=40,  # Changed from num_cells to hidden_size
    dropout_rate=0.1,
    scaling=True,
)

# Train the model
predictor = estimator.train(
    training_data=train_data,
    validation_data=test_data,  # Explicit validation
)

# Generate predictions
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_data,
    predictor=predictor,
    num_samples=100,
)

forecasts = list(forecast_it)
tss = list(ts_it)

# Process predictions
forecast_probs = [f.samples.mean(axis=0).squeeze() for f in forecasts]
threshold = 0.5
binary_preds = [(p > threshold).astype(int) for p in forecast_probs]

# Evaluation
evaluator = Evaluator()
agg_metrics, _ = evaluator(tss, forecasts)

# Classification metrics
actual = data['rainfall'][-k:].values
predicted = np.concatenate(binary_preds)

print("\nProbabilistic Metrics:")
print(f"CRPS: {agg_metrics['mean_wQuantileLoss']:.4f}")
print(f"MAE: {agg_metrics['MAE']:.4f}")

print("\nClassification Metrics:")
print(f"Accuracy: {accuracy_score(actual, predicted):.4f}")
print(f"Precision: {precision_score(actual, predicted):.4f}") 
print(f"Recall: {recall_score(actual, predicted):.4f}")
print(f"F1: {f1_score(actual, predicted):.4f}")

C:\Users\Sathya\AppData\Local\Temp\ipykernel_9688\1868082562.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')


TypeError: DeepARModel.__init__() got an unexpected keyword argument 'trainer_kwargs'

(100, 1)